In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torchvision.models import EfficientNet_B0_Weights
import pandas as pd

# ✅ GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("✅ 사용 중인 GPU:", torch.cuda.get_device_name(0))
else:
    print("❌ GPU를 사용할 수 없습니다. CPU에서 실행됩니다.")

# ✅ 경로 설정
data_dir = "C:/Users/user/OneDrive/Desktop/Resnet18-real/data/category_data"
model_dir = "C:/Users/user/OneDrive/Desktop/Resnet18-real/model"
csv_dir = "C:/Users/user/OneDrive/Desktop/Resnet18-real/csv"
os.makedirs(model_dir, exist_ok=True)
os.makedirs(csv_dir, exist_ok=True)

# ✅ 하이퍼파라미터 설정
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
EPOCHS = 500
LEARNING_RATE = 0.001

# ✅ 데이터 전처리
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ✅ 데이터셋 로드
dataset = datasets.ImageFolder(data_dir, transform=transform)

# ✅ 상위 클래스와 하위 클래스 매핑
category_mapping = {
    "가방.박스.웨건": ["가방.베낭", "박스", "웨건.카트"],
    "공구.액세서리": ["가랜드", "해머"],
    "냉.난방용품": ["난로.히터", "선풍기.써큘레이터", "에어컨", "전기매트.요.방석"],
    "랜턴": [],
    "매트.침낭": ["발포매트", "에어매트", "자충매트", "침낭", "침대", "카페트.러그"],
    "식기.주방": ["물통.워터저그", "세척.건조도구", "식기류", "아이스박스.쿨러", "조리도구"],
    "전기.전자기기": ["TV.모니터.빔프로젝터", "릴선.멀티탭", "스피커", "전동펌프", "파워뱅크.보조배터리"],
    "테이블.체어": ["체어", "테이블"],
    "텐트.타프": ["타프", "텐트"],
    "화로대.버너": ["그릴.그리들", "버너.토치", "화로대"]
}

# ✅ 하위 클래스를 예측하고 상위 클래스를 매핑하기 위한 함수
def map_to_top_class(sub_class, category_mapping):
    for top_class, sub_classes in category_mapping.items():
        if sub_class in sub_classes:
            return top_class
    return None  # 상위 클래스가 없다면 None 반환

# ✅ 상위 클래스 목록
top_classes = list(category_mapping.keys())
num_top_classes = len(top_classes)

# ✅ 데이터셋 분할 (10% 테스트, 나머지 90% 학습/검증)
test_size = int(0.1 * len(dataset))
train_val_size = len(dataset) - test_size
dataset_train_val, dataset_test = random_split(dataset, [train_val_size, test_size])

# ✅ 학습 및 검증 데이터 분할 (80% 학습, 20% 검증)
train_size = int(0.8 * len(dataset_train_val))
valid_size = len(dataset_train_val) - train_size
train_dataset, valid_dataset = random_split(dataset_train_val, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

# ✅ 모델 정의
model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(dataset.classes))  # 하위 클래스 예측
model.to(device)

# ✅ 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ✅ 학습 루프
best_valid_accuracy = 0.0
best_epoch_info = None  # 최적 에포크 정보 저장
csv_path = os.path.join(csv_dir, "category_classification_results.csv")
train_losses, valid_losses, train_accuracies, valid_accuracies = [], [], [], []

for epoch in range(EPOCHS):
    print(f"\n📢 Epoch [{epoch+1}/{EPOCHS}]")
    model.train()
    train_loss, train_correct = 0, 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()
        total_train += labels.size(0)

    train_loss /= total_train
    train_accuracy = train_correct / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # ✅ 검증 단계
    model.eval()
    valid_loss, valid_correct = 0, 0
    total_valid = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * inputs.size(0)
            valid_correct += (outputs.argmax(1) == labels).sum().item()
            total_valid += labels.size(0)
    
    valid_loss /= total_valid
    valid_accuracy = valid_correct / total_valid
    valid_losses.append(valid_loss)
    valid_accuracies.append(valid_accuracy)
    
    print(f"📉 Train Loss: {train_loss:.4f} | 🎯 Train Accuracy: {train_accuracy:.4f} | 📉 Valid Loss: {valid_loss:.4f} | 🎯 Valid Accuracy: {valid_accuracy:.4f}")
    
    # ✅ 최고 검증 정확도 모델 저장
    if valid_accuracy > best_valid_accuracy:
        best_valid_accuracy = valid_accuracy
        best_epoch_info = (epoch+1, train_loss, train_accuracy, valid_loss, valid_accuracy)
        best_model_path = os.path.join(model_dir, "best_category_model.pth")
        torch.save(model.state_dict(), best_model_path)
        print(f"🎯 새로운 최고 모델 저장됨! (Epoch {epoch+1}, Accuracy: {best_valid_accuracy:.4f})")

# ✅ 최적 에포크 정보 출력
if best_epoch_info:
    print("\n🔥 학습 종료! 가장 높은 검증 정확도를 기록한 에포크 🔥")
    print(f"✅ Best Epoch: {best_epoch_info[0]}")
    print(f"📉 Best Train Loss: {best_epoch_info[1]:.4f}")
    print(f"🎯 Best Train Accuracy: {best_epoch_info[2]:.4f}")
    print(f"📉 Best Valid Loss: {best_epoch_info[3]:.4f}")
    print(f"🎯 Best Valid Accuracy: {best_epoch_info[4]:.4f}")

# ✅ 결과 CSV 저장
df = pd.DataFrame({
    'Epoch': range(1, len(train_losses) + 1),
    'Train Loss': train_losses,
    'Valid Loss': valid_losses,
    'Train Accuracy': train_accuracies,
    'Valid Accuracy': valid_accuracies
})
df.to_csv(csv_path, index=False)
print(f"📄 학습 결과 CSV 저장됨: {csv_path}")


✅ CUDA Available: True
✅ 사용 중인 GPU: Quadro RTX 4000

📢 Epoch [1/500]
📉 Train Loss: 0.8684 | 🎯 Train Accuracy: 0.7226 | 📉 Valid Loss: 0.5697 | 🎯 Valid Accuracy: 0.8223
🎯 새로운 최고 모델 저장됨! (Epoch 1, Accuracy: 0.8223)

📢 Epoch [2/500]
📉 Train Loss: 0.4873 | 🎯 Train Accuracy: 0.8419 | 📉 Valid Loss: 0.4848 | 🎯 Valid Accuracy: 0.8523
🎯 새로운 최고 모델 저장됨! (Epoch 2, Accuracy: 0.8523)

📢 Epoch [3/500]
📉 Train Loss: 0.3650 | 🎯 Train Accuracy: 0.8816 | 📉 Valid Loss: 0.5438 | 🎯 Valid Accuracy: 0.8399

📢 Epoch [4/500]
📉 Train Loss: 0.2719 | 🎯 Train Accuracy: 0.9128 | 📉 Valid Loss: 0.5950 | 🎯 Valid Accuracy: 0.8387

📢 Epoch [5/500]
📉 Train Loss: 0.2210 | 🎯 Train Accuracy: 0.9267 | 📉 Valid Loss: 0.5961 | 🎯 Valid Accuracy: 0.8439

📢 Epoch [6/500]
📉 Train Loss: 0.1912 | 🎯 Train Accuracy: 0.9361 | 📉 Valid Loss: 0.5762 | 🎯 Valid Accuracy: 0.8451

📢 Epoch [7/500]
📉 Train Loss: 0.1792 | 🎯 Train Accuracy: 0.9419 | 📉 Valid Loss: 0.5689 | 🎯 Valid Accuracy: 0.8499

📢 Epoch [8/500]
📉 Train Loss: 0.1574 | 🎯 Train Accur

In [7]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torchvision.models import EfficientNet_B0_Weights
import json
import pandas as pd

# ✅ GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("✅ 사용 중인 GPU:", torch.cuda.get_device_name(0))

# ✅ 경로 설정
data_dir = "C:/Users/user/OneDrive/Desktop/Resnet18-real/data/category_data"
model_path = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\model\best_category_model.pth"
csv_dir = "C:/Users/user/OneDrive/Desktop/Resnet18-real/csv"
test_indices_path = os.path.join(csv_dir, "test_indices.json")

# ✅ 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ✅ 데이터셋 로드
dataset = datasets.ImageFolder(data_dir, transform=transform)

# ✅ 테스트 데이터셋 로드 (인덱스를 유지)
if os.path.exists(test_indices_path):
    with open(test_indices_path, "r") as f:
        test_indices = json.load(f)
else:
    test_size = int(0.1 * len(dataset))
    test_indices = torch.randperm(len(dataset)).tolist()[:test_size]
    with open(test_indices_path, "w") as f:
        json.dump(test_indices, f)

dataset_test = torch.utils.data.Subset(dataset, test_indices)
train_val_indices = [i for i in range(len(dataset)) if i not in test_indices]
dataset_train_val = torch.utils.data.Subset(dataset, train_val_indices)

# ✅ 학습 및 검증 데이터셋 분할
train_size = int(0.8 * len(dataset_train_val))
valid_size = len(dataset_train_val) - train_size
train_dataset, valid_dataset = random_split(dataset_train_val, [train_size, valid_size])

# ✅ 데이터 로더 준비
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# ✅ 모델 불러오기
model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, len(dataset.classes))
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()

# ✅ 평가 함수 정의
def evaluate_model(loader, dataset_name):
    correct = 0
    total = 0
    results = []
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = outputs.argmax(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            for i in range(len(labels)):
                image_idx = idx * loader.batch_size + i
                pred_label = dataset.classes[predicted[i].item()]
                true_label = dataset.classes[labels[i].item()]
                if pred_label == true_label:
                    results.append(f"[✅ 정답] 이미지 {image_idx}: 예측={pred_label}, 실제={true_label}")
                else:
                    results.append(f"[❌ 오답] 이미지 {image_idx}: 예측={pred_label}, 실제={true_label}")
    
    accuracy = correct / total
    print(f"✅ {dataset_name} 데이터셋 정확도: {accuracy:.4f}")
    for r in results[:20]:  # 상위 20개 결과 출력 (너무 많아질 수 있음)
        print(r)
    return accuracy, results

# ✅ 평가 수행
print("\n🔥 저장된 모델을 이용한 데이터셋 평가 🔥")
train_accuracy, train_results = evaluate_model(train_loader, "Train")
valid_accuracy, valid_results = evaluate_model(valid_loader, "Valid")
test_accuracy, test_results = evaluate_model(test_loader, "Test")

# # ✅ 결과 CSV 저장
# df = pd.DataFrame({
#     'Dataset': ['Train', 'Valid', 'Test'],
#     'Accuracy': [train_accuracy, valid_accuracy, test_accuracy]
# })
# csv_path = os.path.join(csv_dir, "category_classification_evaluation.csv")
# df.to_csv(csv_path, index=False)
# print(f"📄 평가 결과 CSV 저장됨: {csv_path}")

# ✅ 상세 결과 저장
detail_results_path = os.path.join(csv_dir, "detailed_classification_results2.txt")
with open(detail_results_path, "w", encoding="utf-8") as f:
    f.write("\n".join(train_results + valid_results + test_results))
print(f"📄 상세 평가 결과 저장됨: {detail_results_path}")

✅ CUDA Available: True
✅ 사용 중인 GPU: Quadro RTX 4000

🔥 저장된 모델을 이용한 데이터셋 평가 🔥
✅ Train 데이터셋 정확도: 0.9555
[✅ 정답] 이미지 0: 예측=냉·난방용품, 실제=냉·난방용품
[✅ 정답] 이미지 1: 예측=가방·박스·웨건, 실제=가방·박스·웨건
[✅ 정답] 이미지 2: 예측=테이블·체어, 실제=테이블·체어
[✅ 정답] 이미지 3: 예측=냉·난방용품, 실제=냉·난방용품
[✅ 정답] 이미지 4: 예측=매트·침낭, 실제=매트·침낭
[✅ 정답] 이미지 5: 예측=냉·난방용품, 실제=냉·난방용품
[✅ 정답] 이미지 6: 예측=매트·침낭, 실제=매트·침낭
[✅ 정답] 이미지 7: 예측=매트·침낭, 실제=매트·침낭
[✅ 정답] 이미지 8: 예측=가방·박스·웨건, 실제=가방·박스·웨건
[✅ 정답] 이미지 9: 예측=텐트·타프, 실제=텐트·타프
[✅ 정답] 이미지 10: 예측=매트·침낭, 실제=매트·침낭
[✅ 정답] 이미지 11: 예측=텐트·타프, 실제=텐트·타프
[✅ 정답] 이미지 12: 예측=가방·박스·웨건, 실제=가방·박스·웨건
[✅ 정답] 이미지 13: 예측=가방·박스·웨건, 실제=가방·박스·웨건
[✅ 정답] 이미지 14: 예측=매트·침낭, 실제=매트·침낭
[✅ 정답] 이미지 15: 예측=테이블·체어, 실제=테이블·체어
[✅ 정답] 이미지 16: 예측=매트·침낭, 실제=매트·침낭
[✅ 정답] 이미지 17: 예측=냉·난방용품, 실제=냉·난방용품
[✅ 정답] 이미지 18: 예측=식기·주방, 실제=식기·주방
[✅ 정답] 이미지 19: 예측=가방·박스·웨건, 실제=가방·박스·웨건
✅ Valid 데이터셋 정확도: 0.9612
[✅ 정답] 이미지 0: 예측=냉·난방용품, 실제=냉·난방용품
[✅ 정답] 이미지 1: 예측=전기·전자기기, 실제=전기·전자기기
[✅ 정답] 이미지 2: 예측=랜턴, 실제=랜턴
[✅ 정답] 이미지 3: 예측=공구·액세서리, 실제=공구·액세서리
[✅ 정답] 이미지 4: 예측=화로대·버너, 

In [6]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torchvision.models import EfficientNet_B0_Weights
import json
import pandas as pd

# ✅ GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("✅ 사용 중인 GPU:", torch.cuda.get_device_name(0))

# ✅ 경로 설정
data_dir = "C:/Users/user/OneDrive/Desktop/Resnet18-real/data/category_data"
model_path = r"C:\Users\user\OneDrive\Desktop\Resnet18-real\model\category_model3.pth"
csv_dir = "C:/Users/user/OneDrive/Desktop/Resnet18-real/csv"
test_indices_path = os.path.join(csv_dir, "test_indices.json")

# ✅ 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ✅ 데이터셋 로드
dataset = datasets.ImageFolder(data_dir, transform=transform)

# ✅ 테스트 데이터셋 로드 (인덱스를 유지)
if os.path.exists(test_indices_path):
    with open(test_indices_path, "r") as f:
        test_indices = json.load(f)
else:
    test_size = int(0.1 * len(dataset))
    test_indices = torch.randperm(len(dataset)).tolist()[:test_size]
    with open(test_indices_path, "w") as f:
        json.dump(test_indices, f)

dataset_test = torch.utils.data.Subset(dataset, test_indices)
train_val_indices = [i for i in range(len(dataset)) if i not in test_indices]
dataset_train_val = torch.utils.data.Subset(dataset, train_val_indices)

# ✅ 학습 및 검증 데이터셋 분할
train_size = int(0.8 * len(dataset_train_val))
valid_size = len(dataset_train_val) - train_size
train_dataset, valid_dataset = random_split(dataset_train_val, [train_size, valid_size])

# ✅ 데이터 로더 준비
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# ✅ 모델 불러오기
model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, len(dataset.classes))

# ✅ 모델 가중치만 로드
checkpoint = torch.load(model_path, map_location=device)  # 🔥 저장된 체크포인트 로드
model.load_state_dict(checkpoint["model_state_dict"])  # 🔥 "model_state_dict"만 로드
model.to(device)
model.eval()

# ✅ 평가 함수 정의
def evaluate_model(loader, dataset_name):
    correct = 0
    total = 0
    results = []
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = outputs.argmax(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            for i in range(len(labels)):
                image_idx = idx * loader.batch_size + i
                pred_label = dataset.classes[predicted[i].item()]
                true_label = dataset.classes[labels[i].item()]
                if pred_label == true_label:
                    results.append(f"[✅ 정답] 이미지 {image_idx}: 예측={pred_label}, 실제={true_label}")
                else:
                    results.append(f"[❌ 오답] 이미지 {image_idx}: 예측={pred_label}, 실제={true_label}")
    
    accuracy = correct / total
    print(f"✅ {dataset_name} 데이터셋 정확도: {accuracy:.4f}")
    for r in results[:20]:  # 상위 20개 결과 출력 (너무 많아질 수 있음)
        print(r)
    return accuracy, results

# ✅ 평가 수행
print("\n🔥 저장된 모델을 이용한 데이터셋 평가 🔥")
train_accuracy, train_results = evaluate_model(train_loader, "Train")
valid_accuracy, valid_results = evaluate_model(valid_loader, "Valid")
test_accuracy, test_results = evaluate_model(test_loader, "Test")

# # ✅ 결과 CSV 저장
# df = pd.DataFrame({
#     'Dataset': ['Train', 'Valid', 'Test'],
#     'Accuracy': [train_accuracy, valid_accuracy, test_accuracy]
# })
# csv_path = os.path.join(csv_dir, "category_classification_evaluation.csv")
# df.to_csv(csv_path, index=False)
# print(f"📄 평가 결과 CSV 저장됨: {csv_path}")

# ✅ 상세 결과 저장
detail_results_path = os.path.join(csv_dir, "detailed_classification_results2.txt")
with open(detail_results_path, "w", encoding="utf-8") as f:
    f.write("\n".join(train_results + valid_results + test_results))
print(f"📄 상세 평가 결과 저장됨: {detail_results_path}")

✅ CUDA Available: True
✅ 사용 중인 GPU: Quadro RTX 4000

🔥 저장된 모델을 이용한 데이터셋 평가 🔥
✅ Train 데이터셋 정확도: 0.9738
[✅ 정답] 이미지 0: 예측=공구·액세서리, 실제=공구·액세서리
[✅ 정답] 이미지 1: 예측=화로대·버너, 실제=화로대·버너
[✅ 정답] 이미지 2: 예측=매트·침낭, 실제=매트·침낭
[✅ 정답] 이미지 3: 예측=식기·주방, 실제=식기·주방
[✅ 정답] 이미지 4: 예측=매트·침낭, 실제=매트·침낭
[✅ 정답] 이미지 5: 예측=식기·주방, 실제=식기·주방
[✅ 정답] 이미지 6: 예측=텐트·타프, 실제=텐트·타프
[✅ 정답] 이미지 7: 예측=테이블·체어, 실제=테이블·체어
[✅ 정답] 이미지 8: 예측=가방·박스·웨건, 실제=가방·박스·웨건
[✅ 정답] 이미지 9: 예측=텐트·타프, 실제=텐트·타프
[✅ 정답] 이미지 10: 예측=식기·주방, 실제=식기·주방
[✅ 정답] 이미지 11: 예측=매트·침낭, 실제=매트·침낭
[✅ 정답] 이미지 12: 예측=냉·난방용품, 실제=냉·난방용품
[✅ 정답] 이미지 13: 예측=냉·난방용품, 실제=냉·난방용품
[✅ 정답] 이미지 14: 예측=매트·침낭, 실제=매트·침낭
[✅ 정답] 이미지 15: 예측=매트·침낭, 실제=매트·침낭
[✅ 정답] 이미지 16: 예측=텐트·타프, 실제=텐트·타프
[✅ 정답] 이미지 17: 예측=공구·액세서리, 실제=공구·액세서리
[✅ 정답] 이미지 18: 예측=공구·액세서리, 실제=공구·액세서리
[✅ 정답] 이미지 19: 예측=식기·주방, 실제=식기·주방
✅ Valid 데이터셋 정확도: 0.9728
[✅ 정답] 이미지 0: 예측=냉·난방용품, 실제=냉·난방용품
[✅ 정답] 이미지 1: 예측=매트·침낭, 실제=매트·침낭
[✅ 정답] 이미지 2: 예측=식기·주방, 실제=식기·주방
[✅ 정답] 이미지 3: 예측=냉·난방용품, 실제=냉·난방용품
[✅ 정답] 이미지 4: 예측=냉·난방용품, 실제=냉·난방용품
[✅ 정답]